<참조 : https://www.tensorflow.org/tutorials/text/word2vec?hl=ko>

## Word2Vec

### Skipgram / negative sampling
- bag-of-words 모델 : 주변 문맥이 주어지면 단어를 예측
- skip-gram 모델(n-gram 모델) : 단어가 주어지면 단어의 문맥을 예측  
  
- 스킵 그램 모델의 훈련 목표 : 대상 단어가 주어지면 문맥 단어를 예측할 확률을 최대화

  - $\cfrac{1}{T} \sum_{t=1}^T \, \sum_{-c \le j\le c, \,\, j \not = 0}  \, \, log \, p(w_{t+j} | w_t )$  
    - c : 훈련 컨텍스트의 크기

  - $p \left(w_O | w_I \right) = \cfrac{exp \left(v \prime _{w_O} ^T v_{w_I} \right)}{\sum_{w=1} ^W exp \left(v \prime _w ^T v_{w_I} \right)}$  
    - $v, v \prime$ : 단어의 대상 및 컨텍스트 벡터 표현
    - $W$ : 어휘 크기

### 01. 설정

In [1]:
import io
import re
import string
import tensorflow as tf
import tqdm

from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [31]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

#### 01. 01. 예문 벡터화
- 예문 : The wide road shimmered in the hot sun.

In [4]:
# 토큰화

sentence = 'The wide road shimmered in the hot sum'
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [7]:
# 토큰에서 정수 인덱스로 매핑을 저장하는 어휘

vocab, index = {}, 1    # start indexing from 1
vocab['<pad>'] = 0    # add a padding token

for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1

vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sum': 7}


In [8]:
inverse_vocab = {index:token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sum'}


In [9]:
# 문장을 벡터화
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


#### 01. 02. 한 문장에서 스킵 그램 생성
- tf.keras.preprocessing.sequence 모듈 : 데이터 준비를 단순화하는 유용한 기능 제공
- tf.keras.preprocessing.sequence.skipgrams : [0, vocab_size] 범위의 토큰에서 주어진 window_size, example_sequence 에서 스킵 그램 쌍을 생성

In [10]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0
)
print(len(positive_skip_grams))

26


In [11]:
for target, context in positive_skip_grams[:5]:
    print(f'({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})')

(2, 1): (wide, the)
(3, 4): (road, shimmered)
(2, 3): (wide, road)
(1, 6): (the, hot)
(4, 1): (shimmered, the)


#### 01. 03. 스킵 그램 1개에 대한 음성 샘플링
- skipgrams 함수 : 주어진 창 범위 위로 슬라이드 하여 모든 양의 스킴 그램 쌍을 반환
  - 학습을 위한 음성 샘플로 사용할 추가 스킵 그램 쌍을 생성하려면 어휘에서 임의의 단어를 샘플링해야 합니다  
    
- tf.random.log_uniform_candidate_sampler 함수 : 창에서 주어진 대상 단어에 대해 num_ns 개의 음수 샘플을 샘플링합니다

In [14]:
# Get target and context words for one positive skip-gram
target_word, context_word = positive_skip_grams[0]

# Set the number of negtive samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype='int64'), (1, 1))

negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,    # class that should be sampled as 'positive'
    num_true=1,    # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,    # number of negative context words to sample
    unique=True,    # all the negative samples should be unique
    range_max=vocab_size,    # pick index of the samples from [0, vocab_size]
    seed=SEED,    # seed for reproducibility
    name='negative_sampling'    # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


#### 01. 04. 교육 예제

In [15]:
# Add a dimension so you can use concatenation (on the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concat positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label first context word as 1 (positive) followed by num_ns 0s (negative).
label = tf.constant([1] + [0]*num_ns, dtype='int64')

# Reshape target to shape (1, ) and context and label to (num_ns+1, ).
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [17]:
print(f'target_index    : {target}')
print(f'target_word     : {inverse_vocab[target_word]}')
print(f'context_indices : {context}')
print(f'context_words   : {[inverse_vocab[c.numpy()] for c in context]}')
print(f'label           : {label}')

target_index    : 2
target_word     : wide
context_indices : [1 2 1 4 3]
context_words   : ['the', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [18]:
print('target   : ', target)
print('context  : ', context)
print('label    : ', label)

target   :  tf.Tensor(2, shape=(), dtype=int32)
context  :  tf.Tensor([1 2 1 4 3], shape=(5,), dtype=int64)
label    :  tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


### 02. 모든 단계를 함수 하나로 컴파일

In [21]:
# 스킵 그램 샘플링 테이블
# sampling_table[i] : 데이터 세트에서 i번째로 가장 흔한 단어를 샘플링 할 확률을 나타냅니다.
#     이 함수는 샘플링을 위해 단어 주파수의 zipf 분포를 가정합니다

sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [38]:
# 훈련 데이터 생성

# Generates skip_gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    # Elements of each training example are appended to these lists.
    targets, contexts, labels = [], [], []

    # Build the sampling table for vacab_size tokens.
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    # Iterte over all sequences (sentences) in dataset.
    for sequence in tqdm.tqdm(sequences):

        # Generate positive skip-gram pairs for a swquence (sentence).
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0
        )

        # Iterate over each positive skip-gram pair to produce training examples
        # with positive context word and negative samples.
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                tf.constant([context_word], dtype='int64'), 
                1
            )
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=SEED,
                name='negative_sampling'
            )

            # Build context and label vectors (for one target word)
            negative_sampling_candidates = tf.expand_dims(
                negative_sampling_candidates, 
                1
            )

            context = tf.concat([context_class, negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype='int64')

            # Append each element from the training example to global lists.
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)

    return targets, contexts, labels

### 03. Word2Vec에 대한 데이터 준비

#### 03. 01. text corpus download

In [23]:
# 텍스트 코퍼스 다운로드
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [24]:
with open(path_to_file) as f:
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [25]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

#### 03. 02. 말뭉치에서 문장 벡터화

In [27]:
# Now, create a custom standardization function to lowercase the text and remove punctuation.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                    '[%s]' % re.escape(string.punctuation), '')
    

# Define the vocabulary size and number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the text vectorization layer to normalize, split, and map strings to integers.
# Set output_sequence_length lengnth to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [28]:
# 어휘 생성 : 텍스트 데이터 세트에서 adapt를 호출
vectorize_layer.adapt(text_ds.batch(1024))

In [29]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [32]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

#### 03. 03. 데이터 세트에서 시퀀스 가져오기

In [33]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [34]:
for seq in sequences[:5]:
    print(f'{seq} => {[inverse_vocab[i] for i in seq]}')

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


#### 03. 04. 시퀀스에서 훈련 예제

In [39]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED
)
print(len(targets), len(contexts), len(labels))

100%|██████████| 32777/32777 [00:33<00:00, 971.15it/s] 

65016 65016 65016


#### 03. 04. 성능을 위한 데이터 세트 구성

In [41]:
BATCH_SIZE =1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [42]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


### 04. 모델 및 훈련
- 서브 클래싱된 Word2Vec 모델 : Keras Subclassing API 사용
  - target_embedding : 대상 단어로 표시될 때 단어의 임베딩을 조회하는 tf.keras.layers.Embedding 레이어
    - 매개변수 : (vocab_size * embedding_dim)
  - context_embedding : 문맥 단어로 나타날 때 단어의 임베딩을 조회하는 레이어
    - 매개변수 : target_embedding의 매개변수와 개수 동일합니다
  - dots : 학습 쌍에서 대상 및 컨텍스트 임베딩의 내적을 계산
  - flatten : dots 레이어의 결과를 tf.keras.layers.Flatten으로 병합하는 레이어

In [53]:
class Word2Vec(Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = Embedding(vocab_size,
                                          embedding_dim,
                                          input_length=1,
                                          name='w2v_embedding')
        self.context_embedding = Embedding(vocab_size,
                                           embedding_dim,
                                           input_length=num_ns+1)
        self.dots = Dot(axes=(3, 2))
        self.flatten = Flatten()

    def call(self, pair):
        target, context = pair
        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)
        dots = self.dots([context_emb, word_emb])
        return self.flatten(dots)

- 손실함수 정의 및 모델 컴파일

In [54]:
def custom_loss(x_logit, y_true):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [55]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [56]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [57]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 4s 20ms/step - loss: 1.6082 - accuracy: 0.2332
Epoch 2/20
63/63 [==============================] - 1s 11ms/step - loss: 1.5884 - accuracy: 0.5522
Epoch 3/20
63/63 [==============================] - 1s 12ms/step - loss: 1.5396 - accuracy: 0.5928
Epoch 4/20
63/63 [==============================] - 1s 12ms/step - loss: 1.4566 - accuracy: 0.5648
Epoch 5/20
63/63 [==============================] - 1s 12ms/step - loss: 1.3597 - accuracy: 0.5715
Epoch 6/20
63/63 [==============================] - 1s 12ms/step - loss: 1.2650 - accuracy: 0.5993
Epoch 7/20
63/63 [==============================] - 1s 12ms/step - loss: 1.1766 - accuracy: 0.6340
Epoch 8/20
63/63 [==============================] - 1s 12ms/step - loss: 1.0942 - accuracy: 0.6688
Epoch 9/20
63/63 [==============================] - 1s 12ms/step - loss: 1.0172 - accuracy: 0.7021
Epoch 10/20
63/63 [==============================] - 1s 12ms/step - loss: 0.9454 - accuracy: 0.7316
Epoch 11/

### 05. 조회 / 분석

- get_layer() / get_weights() : 모델에서 가중치를 얻습니다
- get_vocabulary() : 한 줄에 하나의 토큰으로 메타 데이터 파일을 빌드하기 위한 어휘를 제공합니다

In [58]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [59]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue    # skip 0, it's padding.
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + '\n')
    out_m.write(word + '\n')

out_v.close()
out_m.close()

In [60]:
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>